<a href="https://colab.research.google.com/github/A0158/Vehicle-Count/blob/main/Vehicle_Count.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
from time import sleep

width_min=80 #minimum width  of rectangle
height_min=80 #minimum height of retangle

offset=6 #Offset

y_cord=450 #y coordinate of line

frames= 10000 #frames per second

data = []
vehicle= 0


def box(x, y, w, h):
    x1 = int(w / 2)
    y1 = int(h / 2)
    cx = x + x1
    cy = y + y1
    return cx,cy

cap = cv2.VideoCapture('20210325113656.mp4')
# For day video
subtract = cv2.createBackgroundSubtractorKNN()
# For night video
subtract = cv2.bgsegm.createBackgroundSubtractorGMG()
while True:
    ret , frame1 = cap.read()
    temp = float(1/frames)
    sleep(temp) 
    grey = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(grey,(3,3),5)
    img_sub = subtract.apply(blur)
    dilate = cv2.dilate(img_sub,np.ones((5,5)))

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    dilatedata = cv2.morphologyEx (dilate, cv2. MORPH_CLOSE , kernel)
    dilatedata = cv2.morphologyEx (dilatedata, cv2. MORPH_CLOSE , kernel)
    contour,h=cv2.findContours(dilatedata,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    cv2.line(frame1, (600, y_cord), (1500, y_cord), (255,127,0), 3) 
    for(i,c) in enumerate(contour):
        (x,y,w,h) = cv2.boundingRect(c)
        validate_contour = (w >= width_min) and (h >= height_min)
        if not validate_contour:
            continue

        #cv2.rectangle(frame1,(x,y),(x+w,y+h),(0,255,0),2)        
        centre = box(x, y, w, h)
        data.append(centre)
        cv2.circle(frame1, centre, 4, (0, 0,255), -1)

        for (x,y) in data:
            if y<(y_cord+offset) and y>(y_cord-offset):
                vehicle+=1
                cv2.line(frame1, (25, y_cord), (1500, y_cord), (0,127,255), 3)  
                data.remove((x,y))
                print("car is detected : "+str(vehicle))        
       
    cv2.putText(frame1, "VEHICLE COUNT : "+str(vehicle), (450, 70), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255),5)
    cv2.imshow("Video Original" , frame1)
    cv2.imshow("Detectar",dilatedata)

    if cv2.waitKey(1) == 27:
        break
    
cv2.destroyAllWindows()
cap.release()